In [73]:
#vårt projekt

from typing import Tuple

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.utils.data import dataset
from torch.nn.utils.rnn import pad_sequence

import random
import numpy as np
import math

import matplotlib.pyplot as plt
from tqdm import tqdm

In [74]:
#pip install sentencepiece

In [75]:
import sentencepiece as spm

In [76]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

In [77]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
DEVICE = torch.device(device)
DEVICE


device(type='cpu')

In [78]:
#Alice paths
#data_path = '/Users/alicetottie/Downloads/projekt_data/train_paracrawl.en'
#data_path_2 = '/Users/alicetottie/Downloads/projekt_data/train_paracrawl.sv'

#Maja paths
# data_path = '/Users/majarygard/Documents/LTH/Projekt i Data/train_paracrawl.en'
# data_path_2 = '/Users/majarygard/Documents/LTH/Projekt i Data/train_paracrawl.sv'

#Pernilla paths
data_path = 'C:\\Users\\nilla\\Plugg\\project-data\\train_paracrawl.en\\train_paracrawl.en'
data_path_2 = 'C:\\Users\\nilla\\Plugg\\project-data\\train_paracrawl.sv\\train_paracrawl.sv'


In [79]:
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

with open(data_path_2, "r", encoding="utf-8") as f:
    lines2 = f.read().split("\n")    

In [80]:
len(lines)



4960283

In [81]:
len(lines2)

4960283

In [82]:
num_samples = 1000
input_texts = []
target_texts = []

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text= line.split("\t")
    input_texts.append(input_text)

with open("input_texts.txt", "w") as f:
    for input_text in input_texts:
        f.write('\t'.join(input_text) + '\n')
    
for line in lines2[: min(num_samples, len(lines2) - 1)]:
    target_text= line.split("\t")
    target_texts.append(target_text)  

with open("target_texts.txt", "w") as f:
    for target_text in target_texts:
        f.write('\t'.join(target_text) + '\n') 

In [83]:
input_texts[500]



['" This is my third course and I bought this concept a long time ago.']

In [84]:
target_texts[500]

['" Det här är min tredje kurs och jag har köpt det hela för länge sedan.']

In [85]:
TRAIN_PERCENTAGE = 0.8
train_val = int(TRAIN_PERCENTAGE * num_samples)
train_val

800

In [86]:
text_pairs = list(zip(input_texts, target_texts))
random.shuffle(text_pairs)
input_texts, target_texts = zip(*text_pairs)
input_texts, target_texts = list(input_texts), list(target_texts)


In [87]:
train_input_texts = input_texts[:train_val]
train_target_texts = target_texts[:train_val]

val_input_texts = input_texts[train_val:]
val_target_texts = target_texts[train_val:]

In [88]:
spm.SentencePieceTrainer.train('--input=input_texts.txt --model_prefix=m --vocab_size=2000 --model_type=bpe --pad_id=3')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

sp.SetEncodeExtraOptions("bos:eos")
sp.SetDecodeExtraOptions("bos:eos")

# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([1, 665, 64, 5, 4, 134, 2]))

['<s>', '▁This', '▁is', '▁a', '▁t', 'est', '</s>']
[1, 665, 64, 5, 4, 134, 2]
This is a test
This is a test


In [89]:
def text2codes(texts, sp):
    codes = []
    for text in texts:
        text_l = text[0]
        code = sp.encode_as_ids(text_l)
        n_code = torch.tensor(code)
        codes.append(n_code)

    return codes



In [90]:
train_input_texts[:3], train_target_texts[:3]

([["!!TheM!Z!! hasn't added any friends yet!"],
  ['" Memory / Summerday" gouache/wood/butterfly 48x75 / 48x92cm 2006'],
  ['" child safety in Egypt "" harmfulness of the sun in Egypt on baby skin "" what medications the child in Egypt "" where on holiday with his family to Egypt "" pediatrician "" in which the child free hotels "']],
 [['!!TheM!Z!! har inte lagt till några vänner ännu!'],
  ['"Hågkomst / Sommardag" gouache/trä 48x75 /48x92cm2006'],
  ['" barnsäkerhet i Egypten "" skadlighet solen i Egypten på baby hud "" vilka mediciner barnet i Egypten "" där på semester med sin familj till Egypten "" barnläkare "" där barnet hotellsökningen "']])

In [91]:
text2codes(train_input_texts[:3], sp)[:3]

[tensor([   1,  314,  888, 1928, 1938, 1976,  460,  342, 1945, 1900, 1785,  242,
         1041,  304, 1938,    2]),
 tensor([   1,    8,   67,  171,  354,  464,   50,  130,  498,  894, 1911,   54,
           27,   71,    6, 1966, 1428, 1966,  891,  103, 1914,  108,  393, 1962,
         1951, 1957, 1943,  464,  393, 1962, 1951, 1582,  964,  341, 1956,    2]),
 tensor([   1,    8,  224,  396, 1260,  608,   34,  154,   78,   37,  414,  296,
          437,   23,    9, 1263,   34,  154,   89,   25, 1612, 1770,   78,  280,
         1792,  406,    9,  224,  396,   34,  154,   78,  709,   89, 1554,  111,
          246, 1824,   31,  154,   78,   36,   40, 1903,   17, 1636,  431,   78,
           34,  226,    9,  224,  396,   21,  434, 1036,    8,    2])]

In [92]:
def codes2text(codes):
    texts = []
    for code in codes:
        code_l = list(code)
        for p in code_l:
            texts.append(sp.id_to_piece(p.item()))
    return texts

In [93]:
codes2text(text2codes(train_input_texts[:3], sp)[:3])

['<s>',
 '▁!!',
 'The',
 'M',
 '!',
 'Z',
 '!!',
 '▁hasn',
 "'",
 't',
 '▁added',
 '▁any',
 '▁friends',
 '▁yet',
 '!',
 '</s>',
 '<s>',
 '▁"',
 '▁M',
 'em',
 'ory',
 '▁/',
 '▁S',
 'um',
 'mer',
 'day',
 '"',
 '▁g',
 'ou',
 'ac',
 'he',
 '/',
 'wood',
 '/',
 'but',
 'ter',
 'f',
 'ly',
 '▁4',
 '8',
 'x',
 '7',
 '5',
 '▁/',
 '▁4',
 '8',
 'x',
 '92',
 'cm',
 '▁200',
 '6',
 '</s>',
 '<s>',
 '▁"',
 '▁ch',
 'ild',
 '▁saf',
 'ety',
 '▁in',
 '▁Egypt',
 '▁""',
 '▁h',
 'arm',
 'ful',
 'ness',
 '▁of',
 '▁the',
 '▁sun',
 '▁in',
 '▁Egypt',
 '▁on',
 '▁b',
 'aby',
 '▁skin',
 '▁""',
 '▁what',
 '▁medic',
 'ations',
 '▁the',
 '▁ch',
 'ild',
 '▁in',
 '▁Egypt',
 '▁""',
 '▁where',
 '▁on',
 '▁holiday',
 '▁with',
 '▁his',
 '▁family',
 '▁to',
 '▁Egypt',
 '▁""',
 '▁p',
 'ed',
 'i',
 'at',
 'ric',
 'ian',
 '▁""',
 '▁in',
 '▁which',
 '▁the',
 '▁ch',
 'ild',
 '▁f',
 'ree',
 '▁hotels',
 '▁"',
 '</s>']

In [99]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super().__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)
                        * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)
        self.emb_size = emb_size

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding * math.sqrt(self.emb_size)
                            + self.pos_embedding[:token_embedding.size(0), :])

In [100]:
pe = PositionalEncoding(10, 0.1)

In [101]:
torch.zeros(1, 5, 10)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [102]:
pe(torch.zeros(1, 5, 10))

tensor([[[0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 0.0000, 0.0000, 1.1111,
          0.0000, 0.0000],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 0.0000,
          0.0000, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
          0.0000, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 0.0000,
          0.0000, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
          0.0000, 1.1111]]])

In [103]:
# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super().__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)

        self.emb_size = emb_size
        # Same source and target embs Sect. 3.4
        self.embedding = nn.Embedding(
            vocab_size, emb_size, padding_idx=3)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
        # Bias to be compatible with embeddings
        self.generator = nn.Linear(emb_size, vocab_size, bias=False)
        self.generator.weight = self.embedding.weight  # Shared weights Sect. 3.4

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.embedding(src))
        tgt_emb = self.positional_encoding(self.embedding(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
            self.embedding(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
            self.embedding(tgt)), memory,
            tgt_mask)


In [104]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float(
        '-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [105]:
generate_square_subsequent_mask(10)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [106]:
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),
                           device=DEVICE).type(torch.bool)

    src_padding_mask = (src == 3).transpose(0, 1)
    tgt_padding_mask = (tgt == 3).transpose(0, 1).type(torch.float32)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


In [107]:
src = pad_sequence(text2codes(
    train_input_texts[:3], sp), padding_value=3)
tgt = pad_sequence(text2codes(
    train_target_texts[:3], sp), padding_value=3)

In [108]:
train_input_texts[:3], train_target_texts[:3]

([["!!TheM!Z!! hasn't added any friends yet!"],
  ['" Memory / Summerday" gouache/wood/butterfly 48x75 / 48x92cm 2006'],
  ['" child safety in Egypt "" harmfulness of the sun in Egypt on baby skin "" what medications the child in Egypt "" where on holiday with his family to Egypt "" pediatrician "" in which the child free hotels "']],
 [['!!TheM!Z!! har inte lagt till några vänner ännu!'],
  ['"Hågkomst / Sommardag" gouache/trä 48x75 /48x92cm2006'],
  ['" barnsäkerhet i Egypten "" skadlighet solen i Egypten på baby hud "" vilka mediciner barnet i Egypten "" där på semester med sin familj till Egypten "" barnläkare "" där barnet hotellsökningen "']])

In [ ]:
create_mask(src, tgt)